In [1]:
from ibpythonic import ibConnection, message
from ibapi.contract import Contract

from time import sleep, strftime
from datetime import datetime
import pickle
import os
from multiprocessing import Pool

import workers

import pandas as pd
import numpy as np

from tinydb import TinyDB, Query

from tqdm import tqdm_notebook as tqdm

In [2]:
def makeStkContract(contractTuple):
    newContract = Contract()
    newContract.symbol = contractTuple[0]
    newContract.secType = contractTuple[1]
    newContract.exchange = contractTuple[2]
    newContract.currency = contractTuple[3]
    return newContract

In [3]:
bar_data_list = []
busy = False
active_symbol = 0

In [4]:
def my_hist_data_handler(msg):
    global bar_data_list
    bar_data_list.append(msg.bar)

In [5]:
def my_hist_data_end_handler(msg):
    global bar_data_list
    global busy
    global active_symbol
    
    '''
    # Merge collected data into dataframe
    df = pd.DataFrame(columns=('date', 'open', 'high', 'low', 'close', 'volume'))
    for bar in bar_data_list:
        row = {'date': bar.date, 'open': bar.open, 'high': bar.high, 'low': bar.low, 'close': bar.close, 'volume': bar.volume}
        df = df.append(row, ignore_index=True)
    df.loc[:, 'date'] = pd.to_datetime(df.date, format='%Y%m%d') #'%Y%m%d  %H:%M:%S'
    df = df.set_index('date')

    # Write dataframe to csv file
    symbol_db = TinyDB('symbol_db.json')
    my_query = Query()
    result = symbol_db.search(my_query.Id == active_symbol)
    current_symbol = result[0]
    sy = current_symbol['Symbol']
    ex = current_symbol['Exchange']
    df.to_csv('data/' + sy + '_' + ex + '.csv')
    '''
    
    # Merge and write received data to pickle file
    symbol_db = TinyDB('symbol_db.json')
    my_query = Query()
    result = symbol_db.search(my_query.Id == active_symbol)
    current_symbol = result[0]
    sy = current_symbol['Symbol']
    ex = current_symbol['Exchange']
    
    # textfile = open('data/' + sy + '_' + ex + '.txt', 'w+')
    # textfile.write("\n".join(bar_data_list))
    # textfile.close()
    
    with open('data/pickles/' + sy + '_' + ex, 'wb') as fp:
        pickle.dump(bar_data_list, fp)
    
    # write finished info to symbol database
    symbol_db.update({'Status': 'Finished'}, my_query.Id == active_symbol)
    symbol_db.close()
    
    bar_data_list = []
    active_symbol = []
    busy = False

In [6]:
def my_error_handler(msg):
    global bar_data_list
    global busy
    global active_symbol
    
    # write msg info to symbol database
    symbol_db = TinyDB('symbol_db.json')
    my_query = Query()
    # todo: clean message
    symbol_db.update({'Status': msg.errorMsg}, my_query.Id == active_symbol)
    symbol_db.close()
    
    bar_data_list = []
    active_symbol = []
    busy = False

In [7]:
# file = open('symbollist.txt')
# symbols = file.read().splitlines()
# file.close()
# symbols = symbols[:3]

# symbols_df = pd.read_csv('symbol_database.csv', sep=';')
# symbols_df['Status'] = 'Untouched'
# symbols_df = symbols_df.loc[0:7, :]

# symbols = ['DAI', 'BMW', 'SAP', 'BAY']
# symbols = ['TSLA', 'ADBE', 'NVDA', 'AMD', 'INTL']

In [8]:
endtime = strftime('%Y%m%d %H:%M:%S')
start_id = 998
end_id = 2000

In [9]:
con = ibConnection(port=7497)
con.register(my_hist_data_handler, message.historicalData)
con.register(my_hist_data_end_handler, message.historicalDataEnd)
con.register(my_error_handler, message.error)

True

In [10]:
con.connect()
    
for index in tqdm(range(start_id, end_id+1)):
    global busy
    global active_symbol
    
    # Get symbol data from symbol db
    symbol_db = TinyDB('symbol_db.json')
    my_query = Query()
    result = symbol_db.search(my_query.Id == index)
    symbol_db.close()
    current_symbol = result[0]
    
    # Skip symbol if data present
    if current_symbol['Status'] != 'untouched':
        continue
    
    # Create contract and request data
    contractTuple = (current_symbol['Symbol'], 'STK', current_symbol['Exchange'], current_symbol['Currency'])
    stkContract = makeStkContract(contractTuple)
    active_symbol = index
    busy = True
    con.reqHistoricalData(1,stkContract,endtime,"10 Y","1 day","MIDPOINT",1,1, keepUpToDate=False, chartOptions=[])
    
    # Wait for data download and storage
    while busy is True:
        sleep(0.2)
    sleep(0.2)

con.disconnect()

Exception in message dispatch.  Handler 'my_hist_data_end_handler' for 'historicalDataEnd'
Traceback (most recent call last):
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibpythonic\dispatcher.py", line 43, in __call__
    results.append(listener(message))
  File "<ipython-input-5-2020e253d921>", line 29, in my_hist_data_end_handler
    current_symbol = result[0]
IndexError: list index out of range
Exception in message dispatch.  Handler 'my_hist_data_end_handler' for 'historicalDataEnd'
Traceback (most recent call last):
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibpythonic\dispatcher.py", line 43, in __call__
    results.append(listener(message))
  File "<ipython-input-5-2020e253d921>", line 29, in my_hist_data_end_handler
    current_symbol = result[0]
IndexError: list index out of range
Exception in message dispatch.  Handler 'my_hist_data_end_handler' for 'historicalDataEnd'
Traceback (most recent call last):
  File "c:\users\marti\dev\_pyvenv

True

unhandled exception in EReader thread
Traceback (most recent call last):
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\reader.py", line 34, in run
    data = self.conn.recvMsg()
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 99, in recvMsg
    buf = self._recvAllMsg()
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 119, in _recvAllMsg
    buf = self.socket.recv(4096)
OSError: [WinError 10038] An operation was attempted on something that is not a socket


In [15]:
symbol_db = TinyDB('symbol_db.json')
my_query = Query()
result = symbol_db.search(my_query.Id <= 203)
for item in result:
    print(item)
symbol_db.close()

{'Id': 0, 'Symbol': 'BNQF', 'Name': 'Collateralized ETC on RICI Enhanced Gas Oil TR Index', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Market data farm connection is OK:usfarm.nj'}
{'Id': 1, 'Symbol': 'CNB', 'Name': 'Lyxor Euro Corporate Bond Ex Financials UCITS ETF', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 2, 'Symbol': 'UEF5', 'Name': 'UBS ETF - MSCI Emerging Markets Socially Responsible UCITS ETF', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Historical Market Data Service error message:No market data permissions for FWB STK'}
{'Id': 3, 'Symbol': 'GOMA', 'Name': 'BNP Paribas Easy Barclays Euro Government Inflation Linked All Maturities ETF', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Historical Market Data Service error message:No market data permissions for FWB STK'}
{'Id': 4, 'Symbol': 'X0BM', 'Name': 'Coba ETC -2 x Platinum Daily Short Index', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 5, 'Symbol': 'FLOT', 'Name': 

In [ ]:
symbol_db = TinyDB('symbol_db.json')
data_list = str(symbol_db.all())
data_list = data_list.replace("'", '"')
df = pd.read_json(data_list, orient='records')
df

In [57]:
 con.disconnect()

False

In [3]:
counter = 0

for filename in tqdm(os.listdir('data/pickles/')):
    
    if filename.startswith('xxx'):
        continue
    
    with (open('data/pickles/'+filename, 'rb')) as openfile:
        datalist = pickle.load(openfile)
    
    # Merge collected data into dataframe
    df = pd.DataFrame(columns=('date', 'open', 'high', 'low', 'close', 'volume'))
    for bar in datalist:
        row = {'date': bar.date, 'open': bar.open, 'high': bar.high, 'low': bar.low, 'close': bar.close, 'volume': bar.volume}
        df = df.append(row, ignore_index=True)
    df.loc[:, 'date'] = pd.to_datetime(df.date, format='%Y%m%d') #'%Y%m%d  %H:%M:%S'
    df = df.set_index('date')
    
    # Write dataframe to csv file
    df.to_csv('data/' + filename + '.csv')
    
    # Rename pickle file for delting
    os.rename('data/pickles/' + filename, 'data/pickles/' + 'xxx' + filename)
    
    counter += 1
    if counter > 50:
        break

In [17]:
file_list = []
for filename in os.listdir('data/pickles/'):
    file_list.append(filename)
    
if __name__ ==  '__main__': 
    with Pool(3) as p:
        p.map(workers.pick_to_csv, file_list)

# Enhancements

### Speedup
- download next symbol as soon as data is in file

### Extra features for downloading
- start time for downloading is end time of data. last candle of data is considered invalid
- check for splits/mergers

### Symbol in multiple exchanges
- if symbol is multiple times in list
    - load data from primary exchange if ppossible
    - if primary is not available, load from exchange with highest average (price-)volume


### Symbol database
- pull symbols from ib webpage
- pull symbol data from ib
- abillity to create own univestes (=symbol lists) based on symbol data

### Storage of price data
- store price data in database? sqlite, as handeled in one file?
- cloud storage of data

### Misc
- 